In [13]:
import os
import sys
import logging

import pandas as pd

# import seaborn as sns
# sns.set_theme()
# sns.set_color_codes()
import matplotlib.pyplot as plt
import numpy as np

import plotly.express as px
import plotly.io as pio

# pio.renderers.default = "svg"  # comment this line to use interactive plots
import plotly.graph_objects as go


# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
sys.path.append("../components/paddock/")

%load_ext autoreload
%autoreload 2

from telemetry.influx import Influx
from telemetry.analyzer import Analyzer

from IPython.display import Image

pd.set_option("display.max_columns", None)

influx = Influx()
influx.laps_from_file("iracing.csv")
influx.laps_from_file("ferrari.csv")
analyzer = Analyzer()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
(
    game,
    session,
    track,
    car,
    # ) = "iRacing,1674828783,barcelona gp,Ferrari 488 GT3 Evo 2020".split(",")      # circuit
    # ) = "iRacing,1674679321,phoenix rallycross,Volkswagen Beetle GRC ".split(",")  # rallycross
    # ) = "iRacing,1670307078,fiveflags,Chevrolet Monte Carlo SS".split(",")         # oval
    # ) = "iRacing,1673296770,nurburgring nordschleifetourist,Porsche 911 GT3 Cup (991)".split(",")         # long track
) = "iRacing,1681021274,magnycours gp,Ferrari 488 GT3 Evo 2020".split(
    ","
)  # short track
all_laps = influx.telemetry_for(game=game, track=track, car=car)


        from(bucket: "fast_laps")
        |> range(start: 2023-04-09T02:36:58.667557Z, stop: 2023-04-09T02:39:58.239903Z)
        |> filter(fn: (r) => r["_measurement"] == "fast_laps")
        |> filter(fn: (r) => r["SessionId"] == "1681021274")
        |> pivot(rowKey: ["_time"], columnKey: ["_field"], valueColumn: "_value")
        |> filter(fn: (r) => r["CurrentLap"] == "8")
        |> sort(columns: ["_time"], desc: false)
        

        from(bucket: "fast_laps")
        |> range(start: 2023-04-09T15:48:38.468500Z, stop: 2023-04-09T15:52:05.117312Z)
        |> filter(fn: (r) => r["_measurement"] == "fast_laps")
        |> filter(fn: (r) => r["SessionId"] == "1681069716")
        |> pivot(rowKey: ["_time"], columnKey: ["_field"], valueColumn: "_value")
        |> filter(fn: (r) => r["CurrentLap"] == "1")
        |> sort(columns: ["_time"], desc: false)
        

        from(bucket: "fast_laps")
        |> range(start: 2023-04-09T15:52:05.153662Z, stop: 2023-04-09T15:53:44.294714

In [61]:
# Resample to 1 meter intervals
df = all_laps[2].copy()
# remove all Gear == 0 rows
df = df[df["Gear"] != 0]
# display(df)
# df = analyzer.resample(df, freq=1, columns=["Brake", "SpeedMs", "Throttle", "Gear"])
# # normalise SpeedMs and Gear
df["SpeedMsNormalized"] = df["SpeedMs"] / df["SpeedMs"].max()
df["GearNormalized"] = df["Gear"] / df["Gear"].max()

df

,result,table,_start,_stop,_time,CarModel,CurrentLap,GameName,SessionId,SessionTypeName,TrackCode,_measurement,host,topic,user,Brake,Clutch,CurrentLapIsValid,CurrentLapTime,DistanceRoundTrack,Gear,Handbrake,LapTimePrevious,PreviousLapWasValid,Rpms,SpeedMs,SteeringAngle,Throttle,id,SpeedMsNormalized,GearNormalized
0,_result,0,2023-04-09 15:52:05.153662+00:00,2023-04-09 15:53:44.294714+00:00,2023-04-09 15:52:05.411558+00:00,Ferrari 488 GT3 Evo 2020,2,iRacing,1681069716,Qualify,magnycours gp,fast_laps,telegraf,crewchief/Jim/1681069716/iRacing/magnycours gp...,Jim,0.000000,1.0,True,99.224600,0.513433,3.0,0.0,-1.0000,True,0.0,36.414190,0.032830,1.0,1681069716-2,0.537778,0.5
1,_result,0,2023-04-09 15:52:05.153662+00:00,2023-04-09 15:53:44.294714+00:00,2023-04-09 15:52:05.412227+00:00,Ferrari 488 GT3 Evo 2020,2,iRacing,1681069716,Qualify,magnycours gp,fast_laps,telegraf,crewchief/Jim/1681069716/iRacing/magnycours gp...,Jim,0.000000,1.0,True,99.257935,1.715585,3.0,0.0,-1.0000,True,0.0,36.518147,0.010169,1.0,1681069716-2,0.539314,0.5
2,_result,0,2023-04-09 15:52:05.153662+00:00,2023-04-09 15:53:44.294714+00:00,2023-04-09 15:52:05.412996+00:00,Ferrari 488 GT3 Evo 2020,2,iRacing,1681069716,Qualify,magnycours gp,fast_laps,telegraf,crewchief/Jim/1681069716/iRacing/magnycours gp...,Jim,0.000000,1.0,True,99.274600,2.317965,3.0,0.0,-1.0000,True,0.0,36.575443,-0.005521,1.0,1681069716-2,0.540160,0.5
3,_result,0,2023-04-09 15:52:05.153662+00:00,2023-04-09 15:53:44.294714+00:00,2023-04-09 15:52:05.413734+00:00,Ferrari 488 GT3 Evo 2020,2,iRacing,1681069716,Qualify,magnycours gp,fast_laps,telegraf,crewchief/Jim/1681069716/iRacing/magnycours gp...,Jim,0.000000,1.0,True,99.307930,3.526358,3.0,0.0,-1.0000,True,0.0,36.690220,-0.045323,1.0,1681069716-2,0.541855,0.5
4,_result,0,2023-04-09 15:52:05.153662+00:00,2023-04-09 15:53:44.294714+00:00,2023-04-09 15:52:05.414301+00:00,Ferrari 488 GT3 Evo 2020,2,iRacing,1681069716,Qualify,magnycours gp,fast_laps,telegraf,crewchief/Jim/1681069716/iRacing/magnycours gp...,Jim,0.000000,1.0,True,99.341270,4.739051,3.0,0.0,-1.0000,True,0.0,36.812120,-0.080187,1.0,1681069716-2,0.543655,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2407,_result,0,2023-04-09 15:52:05.153662+00:00,2023-04-09 15:53:44.294714+00:00,2023-04-09 15:53:20.684093+00:00,Ferrari 488 GT3 Evo 2020,2,iRacing,1681069716,Qualify,magnycours gp,fast_laps,telegraf,crewchief/Jim/1681069716/iRacing/magnycours gp...,Jim,0.763136,1.0,True,75.542564,3548.329590,3.0,0.0,99.1987,True,0.0,42.074670,-0.103720,0.0,1681069716-2,0.621374,0.5
2408,_result,0,2023-04-09 15:52:05.153662+00:00,2023-04-09 15:53:44.294714+00:00,2023-04-09 15:53:20.698632+00:00,Ferrari 488 GT3 Evo 2020,2,iRacing,1681069716,Qualify,magnycours gp,fast_laps,telegraf,crewchief/Jim/1681069716/iRacing/magnycours gp...,Jim,0.748124,1.0,True,75.575900,3549.721680,3.0,0.0,99.1987,True,0.0,41.499138,-0.114470,0.0,1681069716-2,0.612875,0.5
2409,_result,0,2023-04-09 15:52:05.153662+00:00,2023-04-09 15:53:44.294714+00:00,2023-04-09 15:53:20.732560+00:00,Ferrari 488 GT3 Evo 2020,2,iRacing,1681069716,Qualify,magnycours gp,fast_laps,telegraf,crewchief/Jim/1681069716/iRacing/magnycours gp...,Jim,0.736209,1.0,True,75.609230,3551.094730,3.0,0.0,99.1987,True,0.0,40.927750,-0.121734,0.0,1681069716-2,0.604436,0.5
2410,_result,0,2023-04-09 15:52:05.153662+00:00,2023-04-09 15:53:44.294714+00:00,2023-04-09 15:53:20.762247+00:00,Ferrari 488 GT3 Evo 2020,2,iRacing,1681069716,Qualify,magnycours gp,fast_laps,telegraf,crewchief/Jim/1681069716/iRacing/magnycours gp...,Jim,0.729775,1.0,True,75.625900,3551.774170,3.0,0.0,99.1987,True,0.0,40.640972,-0.124638,0.0,1681069716-2,0.600201,0.5


In [62]:
# Remove rows where the value is the same as the previous row
# df = df[df['DistanceRoundTrack'] != df['DistanceRoundTrack'].shift(1)]

# Check if the value is increasing compared to the previous value
is_increasing = df["DistanceRoundTrack"] > df["DistanceRoundTrack"].shift(1)

# Get the indices where the value is not increasing
not_increasing_indices = is_increasing[is_increasing == False].index.tolist()

print(not_increasing_indices)
df["DistanceRoundTrack"][not_increasing_indices]

[0]


0    0.513433
Name: DistanceRoundTrack, dtype: float64

In [52]:
def lap_fig(df):
    fig = go.Figure()

    fig.add_scatter(
        x=df["DistanceRoundTrack"],
        y=df["SpeedMsNormalized"],
        marker=dict(size=2),
        name="SpeedMsNormalized",
    )

    fig.add_scatter(
        x=df["DistanceRoundTrack"],
        y=df["Brake"],
        marker=dict(size=1),
        name="Brake",
    )

    fig.add_scatter(
        x=df["DistanceRoundTrack"],
        y=df["Throttle"],
        marker=dict(size=1),
        name="Throttle",
    )

    fig.add_scatter(
        x=df["DistanceRoundTrack"],
        y=df["GearNormalized"],
        marker=dict(size=1),
        name="GearNormalized",
    )
    return fig


lap_fig(df).show()

In [33]:
# break the track into segements, where throttle input is below 1
#  if you need to brake in a turn, you will release the throttle
#  if you dont need to brake, you might still release the throttle
#  if it's a full throttle turn, you will not release the throttle
#       - we might identify these by looking at the steering angle


def get_change_indices(df, column, threshold=0.9, below=True):
    X = df[column].values
    if below:
        mask = X <= threshold
    else:
        mask = X >= threshold
    change_indices = np.where(np.diff(mask))[0]
    # if len(change_indices) is odd, then the last value is a change
    if len(change_indices) % 2 == 1:
        print("adding last index")
        change_indices = np.append(change_indices, len(X) - 1)
    return change_indices


def remove_close_indices(indices, threshold=50):
    if len(indices) == 0:
        return indices
    # merge throttle changes that are close together
    new_indices = [
        indices[0],
        indices[1],
    ]
    for i in range(2, len(indices), 2):
        start_i = indices[i]
        end_i = indices[i + 1]
        previous_end = indices[i - 1]
        if start_i - previous_end <= threshold:
            new_indices[-1] = end_i
        else:
            new_indices.append(start_i)
            new_indices.append(end_i)
    return new_indices


throttle_changes = get_change_indices(df, "Throttle", threshold=0.95, below=True)
brake_changes = get_change_indices(df, "Brake", threshold=0.005, below=False)
# display(throttle_changes)
# display(brake_changes)

throttle_changes = remove_close_indices(throttle_changes)
brake_changes = remove_close_indices(brake_changes)
# display(throttle_changes)
# display(brake_changes)


fig = lap_fig(df)

# add a vertical line for each index
for idx in throttle_changes:
    fig.add_vline(
        x=df["DistanceRoundTrack"][idx],
        line_width=1,
        line_dash="dash",
        line_color="green",
    )
for idx in brake_changes:
    fig.add_vline(
        x=df["DistanceRoundTrack"][idx],
        line_width=1,
        line_dash="dash",
        line_color="red",
    )

fig.show()

adding last index


In [34]:
def get_average(df, start_i, end_i, column="Brake", max=True):
    search_df = df[start_i:end_i]

    if max:
        high = abs(round(search_df[column].max(), 2))
        low = high * 0.9
        start = search_df[search_df[column] > low].index.min()
        end = search_df[search_df[column] > low].index.max()
        average = search_df[search_df[column] > low][column].mean()
    else:
        low = abs(round(search_df[column].min(), 2))
        high = low * 1.1
        if low <= 0.1:
            high = 0.1
        start = search_df[search_df[column] < high].index.min()
        end = search_df[search_df[column] < high].index.max()
        average = search_df[search_df[column] < high][column].mean()

    if np.isnan(average):
        average = (high + low) / 2

    return {
        "high": high,
        "low": low,
        "avg_start": start,
        "avg_end": end,
        "average": round(average, 2),
    }


fig = lap_fig(df)
segments = []
for i in range(0, len(throttle_changes), 2):
    start_i = throttle_changes[i]
    end_i = throttle_changes[i + 1]
    # find the closest number in the brake_idx array

    if len(brake_changes):
        # how far is the brake_idx from the start_idx
        brake_i = brake_changes[np.abs(brake_changes - start_i).argmin()]
    else:
        # no brake changes
        brake_i = 999999

    distance = np.abs(brake_i - start_i)
    segment = {
        "type": "brake",
        "seg_start": start_i,
        "seg_end": end_i,
        "color": "yellow",
    }
    if distance > 20:
        # we are not braking in this segment
        segment["type"] = "throttle"
        segment["color"] = "green"
        segment["start"] = start_i
        segment["end"] = end_i

        avg_data = get_average(df, start_i, end_i, column="Throttle", max=False)
        segment |= avg_data
    else:
        # search back 20 meters to find the start of the brake
        search_start = brake_i - 20
        if search_start < 0:
            search_start = 0
        search_df = df[search_start:brake_i]
        min = search_df["Brake"].min()
        brake_start_i = search_df[search_df["Brake"] == min].index.max()
        segment["start"] = brake_start_i

        search_df = df[brake_start_i:end_i]
        brake_end_i = search_df[search_df["Brake"] > min].index.max()
        segment["end"] = brake_end_i

        # find the max brake value
        avg_data = get_average(df, brake_i, end_i)
        segment |= avg_data

    fig.add_shape(
        type="rect",
        xref="x",
        yref="y",
        x0=df["DistanceRoundTrack"][avg_data["avg_start"]],
        y0=avg_data["low"],
        x1=df["DistanceRoundTrack"][avg_data["avg_end"]],
        y1=avg_data["high"],
        line=dict(color="orange", width=2),
    )

    fig.add_shape(
        type="line",
        xref="x",
        yref="y",
        x0=df["DistanceRoundTrack"][avg_data["avg_start"]],
        y0=avg_data["average"],
        x1=df["DistanceRoundTrack"][avg_data["avg_end"]],
        y1=avg_data["average"],
        line=dict(color="red", width=2),
    )

    fig.add_shape(
        type="rect",
        xref="x",
        yref="paper",
        x0=df["DistanceRoundTrack"][segment["seg_start"]],
        y0=0,
        x1=df["DistanceRoundTrack"][segment["seg_end"]],
        y1=1,
        line=dict(color=segment["color"], width=2, dash="dot"),
    )
    # get lowest gear in this segment
    segment["gear"] = df["Gear"][segment["seg_start"] : segment["seg_end"]].min()
    segment["speed"] = df["SpeedMs"][segment["seg_start"] : segment["seg_end"]].min()
    segments.append(segment)
fig.show()
pd.DataFrame(segments)

,type,seg_start,seg_end,color,start,end,high,low,avg_start,avg_end,average,gear,speed
0,throttle,33,309,green,33,309,0.100,0.070,131,131,0.07,1.0,2.399458
1,throttle,394,570,green,394,570,0.100,0.020,561,562,0.04,3.0,36.503871
2,brake,878,1677,yellow,862,1676,0.140,0.126,876,887,0.14,3.0,37.379260
3,throttle,1858,2278,green,1858,2278,0.594,0.540,2277,2277,0.54,1.0,21.318520
4,throttle,2397,2524,green,2397,2524,0.165,0.150,2518,2518,0.15,5.0,53.148559
5,throttle,2752,3221,green,2752,3221,1.045,0.950,2752,3220,1.00,2.0,25.887234
6,throttle,3479,3522,green,3479,3522,1.012,0.920,3479,3521,1.00,4.0,41.851720
7,throttle,3666,4085,green,3666,4085,1.023,0.930,3666,4084,1.00,2.0,26.152756
8,throttle,4275,4282,green,4275,4282,1.034,0.940,4275,4281,0.98,2.0,29.703369
9,throttle,4395,4408,green,4395,4408,1.034,0.940,4395,4407,0.99,2.0,36.178328
